### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
27,42656168,51673,79085,3,0,0,1140,6,2025-05-16 19:32,2025-05-16 19:33,1,0,6477,7,1
28,42660514,40423,79085,51530,0,0,1141,0,2025-05-17 19:35,2025-05-17 19:35,3,0,2142,3,1
29,42410296,89202,79085,1349,17,70,1141,1,2025-05-18 21:59,2025-05-18 21:59,0,5,10938,3,1
30,42660511,46923,79085,51530,0,0,1141,2,2025-05-19 13:30,2025-05-19 13:30,2,0,3499,3,1
31,42660187,76562,79085,51530,0,0,1141,3,2025-05-20 09:30,2025-05-20 09:30,3,0,6452,3,1
32,42410284,79866,79085,1349,18,70,1141,4,2025-05-21 21:59,2025-05-21 21:59,0,0,9142,1,1
33,42660512,46923,79085,51530,0,0,1141,5,2025-05-22 13:30,2025-05-22 13:30,4,0,2616,3,1
34,42661175,103165,79085,3,0,0,1141,6,2025-05-24 01:29,2025-05-24 01:29,0,0,6869,5,1
35,42410272,79085,23755,1349,19,70,1142,1,2025-05-25 21:59,2025-05-25 21:59,2,1,13912,3,1
36,42665202,149122,79085,51530,0,0,1142,2,2025-05-26 19:30,2025-05-26 19:30,0,0,1213,6,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

12

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,51673,33107,34080,1,4,0,0,0,stra 2,9,7,7,42656168
1,79085,37144,36171,0,6,5,4,0,5-4-1a,8,8,7,42656168
0,40423,36653,34250,0,11,1,1,0,hansi,11,8,8,42660514
1,79085,33308,35711,0,3,2,0,0,5-4-1a,7,8,8,42660514
0,89202,0,0,0,0,0,0,0,DEDADANOTOBA,2,0,4,42410296
1,79085,0,0,0,0,0,0,0,4-2-4a,9,10,10,42410296
0,46923,25026,40574,0,11,2,0,0,PALU PUTRA W,11,10,10,42660511
1,79085,44924,29376,2,0,0,0,0,3-4-3SP,8,9,10,42660511
0,76562,29072,36445,6,16,1,0,0,Hardangen.B2P,11,9,12,42660187
1,79085,41434,34061,1,3,5,2,0,5-3-2a,9,9,9,42660187


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1768

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,10520,20,53,17,2,1,93,14,53579093017011,1349
1,23775,20,52,17,1,2,61,20,52541061017001,1349
2,124139,20,39,12,3,5,51,22,39529051012006,1349
3,23755,20,35,11,2,7,55,35,35520055011010,1349
4,122169,20,35,10,5,5,54,35,35519054010005,1349
5,78671,20,32,10,2,8,39,33,32506039010008,1349
6,79866,20,28,7,7,6,41,21,28520041007009,1349
7,79085,20,26,7,5,8,33,27,26506033007004,1349
8,89569,20,21,6,3,11,32,47,21485032006007,1349
9,80293,20,13,3,4,13,22,40,13482022003003,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()